# 유저의 점수를 코사인 유사도를 사용해 분석

- 총합 점수는 비슷하면서, 각 영역에 대한 점수는 제일 다른 유저 탐색

In [1]:
import numpy as np
import pandas as pd

# 점수 1-10점으로 랜덤 생성
# 인원 수 160명
user_grade = pd.DataFrame(np.random.randint(0,11,size=(160,4)) , index=range(1,161), columns=['데이터분석','머신러닝','웹','디자인'])
# 각 유저(사용자)별 네 점수 총합
user_grade_sum = user_grade.sum(axis=1)

display(user_grade)
display(user_grade_sum)

,데이터분석,머신러닝,웹,디자인
1,8,4,1,6
2,1,10,9,8
3,10,10,3,4
4,6,10,9,1
5,8,8,0,3
...,...,...,...,...
156,3,4,1,1
157,8,2,10,3
158,8,2,6,8
159,0,3,10,3


1      19
2      28
3      27
4      26
5      19
       ..
156     9
157    23
158    24
159    16
160    15
Length: 160, dtype: int64

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저와 유저 간의 유사도
user_based_collab = cosine_similarity(user_grade, user_grade)
print(user_based_collab)

[[1.         0.61891187 0.90601032 ... 0.89871703 0.34042861 0.61748469]
 [0.61891187 1.         0.71833643 ... 0.71817563 0.84518919 0.42584502]
 [0.90601032 0.71833643 1.         ... 0.77151675 0.44187582 0.63341872]
 ...
 [0.89871703 0.71817563 0.77151675 ... 1.         0.63921486 0.76932882]
 [0.34042861 0.84518919 0.44187582 ... 0.63921486 1.         0.60620263]
 [0.61748469 0.42584502 0.63341872 ... 0.76932882 0.60620263 1.        ]]


In [3]:
# 1번 입장에서 진행, 각 점수에서 1번점수를 빼고 절대값을 씌워 1번 사용자의 총합과 얼마나 차이나는지 결정
# 0.1을 곱한 이유? 그대로 사용하면 총합점수차의 영향력이 너무 커진다. 임의로 작은 값 부여
grade_subs = np.abs(user_grade_sum - user_grade_sum[1]) * 0.1

display(grade_subs)

1      0.0
2      0.9
3      0.8
4      0.7
5      0.0
      ... 
156    1.0
157    0.4
158    0.5
159    0.3
160    0.4
Length: 160, dtype: float64

In [4]:
# 높은 추천도를 가지려면?
# user_based_collab 즉, 유사도가 낮아야함 (각 영역별 수치가 최대한 달라야 하므로)
# sub 즉, 총점수차가 낮아야함. (차이가 적어야 비슷한 실력이므로)

evaluation_value = user_based_collab[0] + grade_subs

# 해당 값을 최종 지표로 사용한다.
evaluation_value

1      1.000000
2      1.518912
3      1.706010
4      1.344935
5      0.900433
         ...   
156    1.836225
157    1.094897
158    1.398717
159    0.640429
160    1.017485
Length: 160, dtype: float64

In [5]:
# 해당 지표가 낮은 순서대로 index 뽑아옴

top10 = evaluation_value.sort_values()[:10].index.tolist()

# 추천되는 인원의 인덱스 리스트
top10

[112, 15, 44, 81, 19, 68, 118, 159, 25, 137]

In [6]:
# 같은 원리로 인풋 값에 대해 추천시스템 작동

input_num = int(input()) - 1
grade_subs = np.abs(user_grade_sum - user_grade_sum[input_num + 1]) * 0.1
evaluation_value = user_based_collab[input_num] + grade_subs
top10 = evaluation_value.sort_values()[:10].index.tolist()

print(f"<나의 정보>")
print()
print(f"데이터분석 : {user_grade.iloc[input_num]['데이터분석']}, 머신러닝 : {user_grade.iloc[input_num]['머신러닝']}, 웹 : {user_grade.iloc[input_num]['웹']}, 디자인 : {user_grade.iloc[input_num]['디자인']}, 총합 : {user_grade_sum[input_num + 1]}")
print(f"강점 : {user_grade.iloc[input_num].sort_values(ascending=False)[:2].index.tolist()}")
print()

print("================================")
for idx, value in enumerate(top10):
    value = value - 1
    print()
    print(f"<추천 {idx + 1}순위 : {value}번째 학생>")
    print()
    print(f"데이터분석 : {user_grade.iloc[value]['데이터분석']}, 머신러닝 : {user_grade.iloc[value]['머신러닝']}, 웹 : {user_grade.iloc[value]['웹']}, 디자인 : {user_grade.iloc[value]['디자인']}, 총합 : {user_grade_sum[value + 1]}")
    print(f"강점 : {user_grade.iloc[value].sort_values(ascending=False)[:2].index.tolist()}")
    print()
    print("=========================")

5
<나의 정보>

데이터분석 : 8, 머신러닝 : 8, 웹 : 0, 디자인 : 3, 총합 : 19
강점 : ['머신러닝', '데이터분석']


<추천 1순위 : 125번째 학생>

데이터분석 : 2, 머신러닝 : 0, 웹 : 9, 디자인 : 9, 총합 : 20
강점 : ['디자인', '웹']


<추천 2순위 : 140번째 학생>

데이터분석 : 2, 머신러닝 : 2, 웹 : 5, 디자인 : 10, 총합 : 19
강점 : ['디자인', '웹']


<추천 3순위 : 80번째 학생>

데이터분석 : 0, 머신러닝 : 5, 웹 : 7, 디자인 : 7, 총합 : 19
강점 : ['디자인', '웹']


<추천 4순위 : 58번째 학생>

데이터분석 : 4, 머신러닝 : 1, 웹 : 10, 디자인 : 5, 총합 : 20
강점 : ['웹', '디자인']


<추천 5순위 : 57번째 학생>

데이터분석 : 1, 머신러닝 : 2, 웹 : 5, 디자인 : 10, 총합 : 18
강점 : ['디자인', '웹']


<추천 6순위 : 14번째 학생>

데이터분석 : 3, 머신러닝 : 6, 웹 : 10, 디자인 : 0, 총합 : 19
강점 : ['웹', '머신러닝']


<추천 7순위 : 111번째 학생>

데이터분석 : 0, 머신러닝 : 9, 웹 : 8, 디자인 : 2, 총합 : 19
강점 : ['머신러닝', '웹']


<추천 8순위 : 43번째 학생>

데이터분석 : 0, 머신러닝 : 5, 웹 : 10, 디자인 : 2, 총합 : 17
강점 : ['웹', '머신러닝']


<추천 9순위 : 33번째 학생>

데이터분석 : 4, 머신러닝 : 1, 웹 : 8, 디자인 : 7, 총합 : 20
강점 : ['웹', '디자인']


<추천 10순위 : 158번째 학생>

데이터분석 : 0, 머신러닝 : 3, 웹 : 10, 디자인 : 3, 총합 : 16
강점 : ['웹', '디자인']

